## mnist.yml

In [76]:
%%writefile  mnist_demo/mnist.yml 

---


model:
                                
  - input: images                # build input layer, with name: images, images also is input_x

  - convolution:
      kernels: 64
      size: [2, 2]
  - activation: relu

  - convolution:
      kernels: 96
      size: [2, 2]                # QUESTION: as long as padding=same, convol images won't shrink? 
  - activation: relu
                                  # QUESTION: when pool shrink the image size, I don't need to track size here
  - pool: [3, 3]                  # add a pooling layer with shape 3x3

  - convolution:
      kernels: 96
      size: [2, 2]
  - activation: relu

  - flatten:
  - dense: [64, 10]

  - activation: softmax            # apply softmax activation to output layer
    name: labels                   # Question and Answer: make this layer to be output layer, name it `labels`
                                   # `name: labels` can be replaced with `- output: labels` 


include: mnist_defaults.yml  # to define default setting 
...

Writing mnist_demo/mnist.yml


## mnist_defaults.yml

In [103]:
%%writefile mnist_demo/mnist_defaults.yml

---
                         # Declare the loss function that is used during training/validation/testing.
loss:
                         # It is a list of loss functions, one for each model output.
                         # The MNIST example only has one output, named "labels".
  - target: labels
    name: categorical_crossentropy

                         # The "include" section in "mnist.yml" is magical, and will merge this section
                         # into the section in "mnist.yml".
                         ######## merge: this entire mnist-default.yml, not a few lines here? ##############
                    
                    
                                # See what included in train section: http://kur.deepgram.com/specification.html#train
train:
    
  data:  
    - mnist:                    # mnist is data supplier, with 2 parameters: images, and labels
        images:                 # dataset under variable name images, will be downloaded inside ~/kur
          url: "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz"
          checksum: 440fcabf73cc546fa21475e81ea370265605f56be210a4024d2ca8f203523609
          path: "~/kur/mnist"  # QUESTION!: mnist dataset will be downloaded to `~/kur`, regardless where I set the path?
            
            
            
                                # QUESTION!: Is it possible to use mnist data supplier for other image datasets?
                                # for example, I have local files `images.gz` and `labels.gz` saved in ~/kur
                                # can I use mnist data supplier for my lcoal files? 
                                # how should I create `label.gz`?
                            
                                # QUESTION: STANDARD PACKAGE (http://kur.deepgram.com/specification.html#standard-packaging)
                                # `url`, `checksum`, `path` are universal to other data suppliers, but it up to other data
                                # suppliers to decide to utilize them or not, right?
                                # data suppliers: mnist, cifar, pickle, numpy_dict, csv, speech_recognition, are all different
            
                                # QUESTION: be cautious on file name, key name, input_layer name, output_layer name
                                # read (https://hyp.is/P5qm7ALoEeecumOcfUu09g/kur.deepgram.com/specification.html)
            
            
            
        labels:                 # dataset under variable name labels, will be downloaded inside ~/kur
          url: "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"
          checksum: 3552534a0a558bbed6aed32b30c495cca23d567ec52cac8be1a0730e8010255c
          path: "~/kur/mnist"

            
                                 # how to use see (http://kur.deepgram.com/specification.html#provider)
                                 # As we discuss in "Examples" in the documentation, we only train on the first
                                 # batches each epoch. This is just to make getting through the MNIST example
                                 # nice and quick on slow/CPU-only machines. If you have a GPU, feel free to
                                 # remove the "provider:" section entirely.
                                
  provider:                      
    name: batch_provider         # if missing, then default is `name: batch_provider`, 
    batch_size: 10               # if missing, then `batch_size: 32` is default
    num_batches: 10              # if missing, use all batches, or set any number below max_num_batch
    randomize: true              # if missing, default is true, yes 
        
                                 # more parameters of `provider`: sortagrad, sort_by, shuffle_after, forced_batch_size
                                 # QUESTION!: on the use of `sort_by`
                                 # `sort_by: X` X must be input name or a column name? 
                                 # how and when do we need to use it other than speech example?



  weights: 
    initial: mnist_demo/best_w
    last: mnist_demo/last_w
        
        
  log:                           # QUESTION: see (http://kur.deepgram.com/specification.html#log)
    path: mnist_demo/mnist_log              
                                 # if missing or `log: ` or `log: null`, then nothing is created or saved
                                 # there is more advanced way of using it
        
        
        
                                 # QUESTION: see (http://kur.deepgram.com/specification.html#epochs)
  epochs:                        # if missing, or `epochs: ` or `epochs: null` or `epochs: infinite`, 
                                 # then keep iterating until ctrl + c to stop
    number: 2                    # it can be empty, `null`, `infinite` or just a number
    mode: additional                  
                                 # 2 modes: default is `additional`, the other is `total` with log exist
                                 # mode: can not be empty, but can be missing
    
    
                                 # QUESTION: see (http://kur.deepgram.com/specification.html#optimizer)
                                 # we have options: adam, sgd, rsmprop, with clip 
  optimizer: 
    name: adam
    learning_rate: 0.001
    
    
                                # Attention: checkpoint (http://kur.deepgram.com/specification.html?highlight=sort_by#checkpoints)
                                # every 2 epochs, 5 batches 50 samples, or 30 minutes (`minutes: 30`), whichever is earlist, then
                                # it should pause training, save the weights (path: checkpoint.kur), 
                                # and run a validation run on 100 batches (validation: 100). You can also use `validation: yes` to 
                                # run validation on the entire validation set, 
                                # or `validation: ` no to only save the weights and then resume training.
                            
                                # QUESTION!: given `validation: 100`, will the best weights wrt validation samples all the way up
                                # to the moment of checkpoint, be selected to save inside checkpoint.kur, instead of the latest 
                                # weights up to checkpoint? will each checkpoint be stored so there are many models saved in the end?
                                # or only the latest model will be stored?
                                # log: save the loss for each epoch, record history; 
                                # best_w update the best weights wrt validation samples, record no history;
                                # checkpoint: should record history, right? every model at every checkpoint, should be saved?
                                # However, in this example, checkpoint.kur only has just one model(all layers) weights and biases
  checkpoint:
    path: checkpoint.kur
    epochs: 2
    batches: 5
    samples: 50
    minutes: 30
    validation: 100                                
    
    
    

                         # Here, we use the MNIST test set as a validation set (more generally, you'll
                         # want train, validation, and test sets; but we ignore this for the MNIST
                         # example). The funky "&validation" is just a YAML anchor, so we can reference
                         # this section later.
validate: &validate
  data:
    - mnist:
        images:          #QUESTION AND ANSWER
                         # the checksum is optional. If you need to generate the checksum (because, for example, 
                         # you want to use a new/different dataset, same images and labels with different datasets for train/valid), 
                         # then just run sha256sum FILE (on Linux) or shasum -a 256 FILE (on macOS).
          url: "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz"
          checksum: 8d422c7b0a1c1c79245a5bcf07fe86e33eeafee792b84584aec276f5a2dbc4e6
          path: "~/kur/mnist"
        labels:
          url: "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
          checksum: f7ae60f92e00ec6debd23a6088c31dbd2371eca3ffa0defaefb259924204aec6
          path: "~/kur/mnist"

                         # Let's also use less data for validation, just to speed it along.
  provider:
    num_batches: 5
    batch_size: 8
    randomize: yes

        
                         # ATTENTION: where to save best validation weights see (http://kur.deepgram.com/specification.html#id1)
                         # Don't save if missing, or `weights: ` or `weights: null`
                         # save, with short form: `weights: path`
  weights: 
    best: mnist_demo/best_w


                           # QUESTION!: how to make output hook useful in validate section?
                           # "If in validate section, the hooks are passed a single batch of model output after each validation run. 
                           # This is useful for printing out some examples of your model’s progress."
                           # does it mean: each epoch, a batch of model output will be generated; and it will be very helpful
                           # at end of each epoch, instead of loading from pickle file, such output is automatically printed out 
                           # in a nice readable format for users to see, right? Like we discussed previously about making 
                           # hooks for plotting weights and activations 
  hooks: 
    - output: 
        path: mnist_demo/valid_results.pkl
        format: pickle  # we can have one format for output right?
        
                        # QUESTION: Don't we need separate dataset from training set and validation set?
                        # can Kur perform data split into training, validation and test sets?
        
        
        
        
                         # Let's define the test set, used if you execute something like:
                         # `kur test mnist.yml`
                         # The funky "*validation" is just a YAML alias, so we basically are setting the
                         # "test" section" to be the same as the "validate" section.
                         # this is why `validate: &validate` is very necessary and useful ##################
test: 
  <<: *validate

                         # This is the evaluation section, used during `kur evaluate mnist.yml`.
                         # The funky "<<: *validate" is just YAML, and basically means "copy all of the
                         # keys from 'validate', and then add/replace the 
                         # what does "copy all the keys from 'validate' mean? and "add/replace" what? ######
                         # but how exactly *validate and <<: *validate differ? #############################
           


  weights:               # best weights must set for test too, otherwise, meaningless
    initial: mnist_demo/best_w
        
  provider: 
    num_batches: 30
                        
                        
                         # Use the entire testing set for evaluation, or set any number you like   
evaluate:
  <<: *validate
  


  provider:
    num_batches: null
        
        
  weights:               
    initial: mnist_demo/best_w

        
                         # QUESTION!: with destination, this is to save/update an ouput whenever we run `kur evaluate file`, right?   
                         # (http://kur.deepgram.com/specification.html?highlight=sort_by#destination)
                    
                         # In this example: 
                         # entire test set is used to produce the output pickle file
                         # the pickle file contains 2 dicts, one for prediction labels, another for truth labels
  destination: mnist_demo/mnist.results.pkl

                         # This is a list of post-processing hooks. Here, we want to produce the
                         # digit-by-digit accuracy table (just called "mnist").
                         # QUESTION!: What exactly is such "digit-by-digit accuracy table"? 
                         # How to access this table? How to see it?
  hooks:
    - mnist
...


Overwriting mnist_demo/mnist_defaults.yml


## Create checksum for dataset
- same input or output name, but different sets of data
- use checksum to differentiate them

In [20]:
!shasum -a 256 /Users/Natsume/kur/train-images-idx3-ubyte.gz

440fcabf73cc546fa21475e81ea370265605f56be210a4024d2ca8f203523609  /Users/Natsume/kur/train-images-idx3-ubyte.gz


## Make a third mnist.yml to include

In [104]:
%%writefile mnist_demo/mnist_sim.yml

---
train: 
  epochs: 2

    
    
include: mnist.yml
...



Overwriting mnist_demo/mnist_sim.yml


In [105]:
!kur -v train mnist_demo/mnist_sim.yml

[INFO 2017-03-07 17:24:51,590 kur.kurfile:699] Parsing source: mnist_demo/mnist_sim.yml, included by top-level.
[INFO 2017-03-07 17:24:51,593 kur.kurfile:699] Parsing source: mnist.yml, included by mnist_demo/mnist_sim.yml.
[INFO 2017-03-07 17:24:51,599 kur.kurfile:699] Parsing source: mnist_defaults.yml, included by mnist_demo/mnist.yml.
[INFO 2017-03-07 17:24:51,627 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-07 17:24:51,654 kur.loggers.binary_logger:71] Loading log data: mnist_demo/mnist_log
[INFO 2017-03-07 17:24:52,660 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-07 17:24:52,660 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-07 17:24:52,660 kur.backend.keras_backend:122] No particular backend for Keras has been requested.
[INFO 2017-03-07 17:24:53,800 kur.backend.keras_backend:195] Keras is loaded. The backend is: theano
[INFO 2017-03-07 17:24:53,800 kur.model.model:260] Enumerating the model containers.
[INFO 2017-03-07 17:24:53,800 kur.model.m

## Model builds or not

In [31]:
!kur -vv build mnist_demo/mnist.yml

[INFO 2017-03-07 13:31:51,421 kur.kurfile:699] Parsing source: mnist_demo/mnist.yml, included by top-level.
[INFO 2017-03-07 13:31:51,430 kur.kurfile:699] Parsing source: mnist_defaults.yml, included by mnist_demo/mnist.yml.
[INFO 2017-03-07 13:31:51,450 kur.kurfile:82] Parsing Kurfile...
[DEBUG 2017-03-07 13:31:51,451 kur.kurfile:784] Parsing Kurfile section: settings
[DEBUG 2017-03-07 13:31:51,451 kur.kurfile:784] Parsing Kurfile section: train
[DEBUG 2017-03-07 13:31:51,455 kur.kurfile:784] Parsing Kurfile section: validate
[DEBUG 2017-03-07 13:31:51,460 kur.kurfile:784] Parsing Kurfile section: test
[DEBUG 2017-03-07 13:31:51,464 kur.kurfile:784] Parsing Kurfile section: evaluate
[DEBUG 2017-03-07 13:31:51,470 kur.containers.layers.placeholder:63] Using short-hand name for placeholder: images
[DEBUG 2017-03-07 13:31:51,471 kur.containers.layers.placeholder:97] Placeholder "images" has a deferred shape.
[DEBUG 2017-03-07 13:31:51,473 kur.kurfile:784] Parsing Kurfile section: loss
[I

## Just train 2 epochs

In [78]:
!kur -v train mnist_demo/mnist.yml

[INFO 2017-03-07 14:53:31,525 kur.kurfile:699] Parsing source: mnist_demo/mnist.yml, included by top-level.
[INFO 2017-03-07 14:53:31,537 kur.kurfile:699] Parsing source: mnist_defaults.yml, included by mnist_demo/mnist.yml.
[INFO 2017-03-07 14:53:31,574 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-07 14:53:31,613 kur.loggers.binary_logger:107] Log does not exist. Creating path: mnist_demo/mnist_log
[INFO 2017-03-07 14:53:33,028 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-07 14:53:33,028 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-07 14:53:33,028 kur.backend.keras_backend:122] No particular backend for Keras has been requested.
[INFO 2017-03-07 14:53:34,075 kur.backend.keras_backend:195] Keras is loaded. The backend is: theano
[INFO 2017-03-07 14:53:34,075 kur.model.model:260] Enumerating the model containers.
[INFO 2017-03-07 14:53:34,075 kur.model.model:265] Assembling the model dependency graph.
[INFO 2017-03-07 14:53:34,075 kur.model.model:280

In [79]:
!kur -v evaluate mnist_demo/mnist.yml

[INFO 2017-03-07 14:53:54,808 kur.kurfile:699] Parsing source: mnist_demo/mnist.yml, included by top-level.
[INFO 2017-03-07 14:53:54,817 kur.kurfile:699] Parsing source: mnist_defaults.yml, included by mnist_demo/mnist.yml.
[INFO 2017-03-07 14:53:54,840 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-07 14:53:55,039 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-07 14:53:55,039 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-07 14:53:55,039 kur.backend.keras_backend:122] No particular backend for Keras has been requested.
[INFO 2017-03-07 14:53:56,023 kur.backend.keras_backend:195] Keras is loaded. The backend is: theano
[INFO 2017-03-07 14:53:56,024 kur.model.model:260] Enumerating the model containers.
[INFO 2017-03-07 14:53:56,024 kur.model.model:265] Assembling the model dependency graph.
[INFO 2017-03-07 14:53:56,024 kur.model.model:280] Connecting the model graph.
[INFO 2017-03-07 14:53:57,298 kur.model.model:284] Model inputs:  images
[INFO 2017-03-

In [84]:
!kur -v test mnist_demo/mnist.yml

[INFO 2017-03-07 14:59:37,665 kur.kurfile:699] Parsing source: mnist_demo/mnist.yml, included by top-level.
[INFO 2017-03-07 14:59:37,674 kur.kurfile:699] Parsing source: mnist_defaults.yml, included by mnist_demo/mnist.yml.
[INFO 2017-03-07 14:59:37,695 kur.kurfile:82] Parsing Kurfile...
[INFO 2017-03-07 14:59:37,889 kur.backend.backend:80] Creating backend: keras
[INFO 2017-03-07 14:59:37,892 kur.backend.backend:83] Backend variants: none
[INFO 2017-03-07 14:59:37,892 kur.backend.keras_backend:122] No particular backend for Keras has been requested.
[INFO 2017-03-07 14:59:39,040 kur.backend.keras_backend:195] Keras is loaded. The backend is: theano
[INFO 2017-03-07 14:59:39,040 kur.model.model:260] Enumerating the model containers.
[INFO 2017-03-07 14:59:39,040 kur.model.model:265] Assembling the model dependency graph.
[INFO 2017-03-07 14:59:39,040 kur.model.model:280] Connecting the model graph.
[INFO 2017-03-07 14:59:40,941 kur.model.model:284] Model inputs:  images
[INFO 2017-03-

## Check loss over epochs

In [59]:
from kur.loggers import BinaryLogger
data = BinaryLogger.load_column('mnist_demo/mnist_log/', 'training_loss_labels')

In [60]:
type(data)

numpy.ndarray

In [61]:
data

array([ 2.35232759,  1.52068317], dtype=float32)

## Check evaluate_result.pkl

In [88]:
import pickle

In [89]:
objects = []
with (open("mnist_demo/mnist.results.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [96]:
objects[0]['result']

{'labels': array([[  6.34934171e-04,   1.61952619e-03,   4.64436976e-04, ...,
           9.67636704e-01,   6.90784678e-03,   9.98236984e-03],
        [  2.50942772e-03,   1.99295226e-02,   3.06665897e-02, ...,
           1.41713594e-04,   4.14029285e-02,   2.23570954e-04],
        [  5.93420700e-04,   9.16382194e-01,   5.09680249e-03, ...,
           9.34527908e-03,   2.21553724e-02,   2.46891868e-03],
        ..., 
        [  3.83175304e-03,   1.99483801e-02,   9.33283358e-04, ...,
           1.60496131e-01,   2.52854586e-01,   1.92379564e-01],
        [  4.16923352e-02,   4.19610478e-02,   4.21239715e-03, ...,
           3.33479494e-02,   3.46194357e-01,   4.40075547e-02],
        [  1.28660694e-01,   1.02628364e-04,   3.26921465e-04, ...,
           3.14452591e-05,   9.28878610e-04,   5.14979765e-04]], dtype=float32)}

In [98]:
objects[0]['truth']

{'labels': array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])}

In [99]:
len(objects[0]['result']['labels'])

10000

## Check validate_output.pkl

In [ ]:
mnist_demo/valid_results.pkl

In [106]:
objects = []
with (open("mnist_demo/valid_results.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [107]:
objects

[{'result': {'labels': array([[  8.91884192e-05,   9.22899187e-01,   1.07372273e-02,
             4.75795940e-03,   4.09801403e-04,   1.20847908e-04,
             4.51875059e-03,   1.23824514e-02,   4.35064435e-02,
             5.78166742e-04],
          [  2.28239316e-02,   4.44366684e-04,   1.05896099e-02,
             3.18597704e-02,   5.73459780e-04,   6.58386350e-01,
             1.11939088e-01,   4.87184967e-04,   1.58910364e-01,
             3.98587994e-03],
          [  5.57961240e-02,   7.60489493e-04,   2.80040074e-02,
             4.02259966e-03,   8.62911567e-02,   1.72598765e-03,
             6.85843751e-02,   1.08213490e-02,   5.65326273e-01,
             1.78667635e-01],
          [  4.09069180e-04,   6.24194920e-01,   1.45504475e-01,
             3.56709734e-02,   8.88562412e-04,   1.62830736e-04,
             1.81097556e-02,   4.33615036e-03,   1.70211062e-01,
             5.12199127e-04],
          [  4.35693510e-05,   2.00781171e-04,   1.79562555e-03,
             7.

In [109]:
len(objects[0]['result']['labels'])

8

In [110]:
len(objects[0]['truth']['images'])

8